<a href="https://colab.research.google.com/github/AugustvonMackensen/AI_colab/blob/main/ResNet_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[1] import the packages

In [1]:
from tensorflow import keras
from keras import layers, activations, Input
from keras.datasets import cifar10
import cv2
import numpy as np

[2] prepare the data and pre-processing

In [2]:
def load_cifar10_data(img_rows, img_cols):
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()
  x_train = x_train[0:2000, :, :, :]
  y_train = y_train[0:2000]

  x_test = x_test[0:500, :, :, :]
  y_test = y_test[0:500]

  x_train = np.array([cv2.resize(img, (img_rows, img_cols)) for img in x_train[:, :, :, :]])
  x_test = np.array([cv2.resize(img, (img_rows, img_cols)) for img in x_test[:, :, :, :]])

  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train = x_train / 255.0
  x_test = x_test / 255.0
  return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = load_cifar10_data(224, 224)
img_input = Input(shape=(224,224,3))

170498071/170498071 [==============================] - 4s 0us/step


[3] Define ResidualUnit Class

In [3]:
class ResidualUnit(layers.Layer):
  def __init__(self, filters, strides=1, activation='relu', **kwargs):
    super().__init__(**kwargs)
    self.activation = activations.get(activation)
    self.main_layers = [
        layers.Conv2D(filters, (3,3), strides=strides, padding='same', use_bias=False),
        layers.BatchNormalization(),
        self.activation,
        layers.Conv2D(filters, (3,3), strides=1, padding='same', use_bias=False),
        layers.BatchNormalization()
    ]
    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
          layers.Conv2D(filters, (1,1), strides=strides, padding='same', use_bias=False),
          layers.BatchNormalization()
      ]

  def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
      Z = layer(Z)
    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)
    return self.activation(Z + skip_Z)

[4] Define the Model

In [4]:
model = keras.models.Sequential()
model.add(layers.Conv2D(64, (7,7), strides=(2,2), input_shape=[224, 224, 3], padding='same', use_bias=False))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same'))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
  strides = 1 if filters == prev_filters else 2
  model.add(ResidualUnit(filters,strides=strides))
  prev_filters = filters

model.add(layers.GlobalAvgPool2D())
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

[5] Fit

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
63/63 [==============================] - 42s 218ms/step - loss: 2.5471 - accuracy: 0.2115 - val_loss: 37.0348 - val_accuracy: 0.0820
Epoch 2/10
63/63 [==============================] - 12s 183ms/step - loss: 1.9045 - accuracy: 0.3070 - val_loss: 2.9421 - val_accuracy: 0.1800
Epoch 3/10
63/63 [==============================] - 12s 191ms/step - loss: 1.7811 - accuracy: 0.3365 - val_loss: 3.0029 - val_accuracy: 0.1320
Epoch 4/10
63/63 [==============================] - 12s 192ms/step - loss: 1.7185 - accuracy: 0.3655 - val_loss: 2.2232 - val_accuracy: 0.2760
Epoch 5/10
63/63 [==============================] - 12s 193ms/step - loss: 1.6068 - accuracy: 0.3950 - val_loss: 2.0734 - val_accuracy: 0.2660
Epoch 6/10
63/63 [==============================] - 12s 197ms/step - loss: 1.5376 - accuracy: 0.4340 - val_loss: 2.2120 - val_accuracy: 0.2740
Epoch 7/10
63/63 [==============================] - 12s 187ms/step - loss: 1.4882 - accuracy: 0.4505 - val_loss: 4.4462 - val_accuracy: 0.190

[6] Check the accuracy rate

In [6]:
score = model.evaluate(x_test, y_test)
print('test accuracy rate : ', score[1])

16/16 [==============================] - 1s 59ms/step - loss: 5.0107 - accuracy: 0.2040
test accuracy rate :  0.20399999618530273
